In [10]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from functools import reduce
import matplotlib.cm as cm

In [11]:
L = 2 # Number of qubits
d = 2**L # Dimension of Hilbert space

In [12]:
psi = np.array([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)])
rho = np.outer(psi, psi)

In [13]:
# Define Pauli Matrices
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
paulis = np.array([X, Y, Z])

def get_projectors(measure_settings: np.ndarray):
    """
    Parameters:
        measure_settings: m by L array of 0, 1, or 2 determining Pauli basis
            where m is number of measurement settings
    Returns:
        Length md list of random Pauli basis projectors,
        i.e. returns list of P_k for eigenspace projectors P_k
    """
    eigvecs = np.array([ np.linalg.eig(sigma)[1] for sigma in paulis ])
    # Number of qubits
    m = measure_settings.shape[0]
    d = 2**measure_settings.shape[1] # d = 2^L

    # Set of dxd projectors for each basis for each measurement setting
    P = np.zeros((m, d, d, d), dtype="complex128")

    for j, pauli_idx in enumerate(measure_settings):

        pauli_eigs_l = eigvecs[pauli_idx]

        # Get all combinations of 1 eigenvector from each pauli across L paulis
        eigs_sets = list(product(*pauli_eigs_l)) # 2^L total

        for k, set in enumerate(eigs_sets):
            # Tensor product all eigenvectors
            v_k = reduce(np.kron, set)
            # Form projector onto eigenspace
            proj_k = np.outer(np.conj(v_k), v_k)
            P[j, k, :, :] = proj_k

    return P

In [14]:
def sampling_operator(U: np.ndarray, Pk: np.ndarray):
    """
    Parameters:
        U: dxd density matrix
        Pk: m x d x d x d array of projection operators
    Returns:
        A(U) = [ [tr(UP_1), ...], ..., [..., tr(UP_md)] ]
    """
    return np.array([[
            np.real(np.trace(P @ U)) for P in P_setting ]
        for P_setting in Pk ])

In [15]:
N = 100 # Num. of measurement repeats
m = 11 # Num. of measurement settings
measures_test = np.random.randint(0, 3, size=(m, L))

In [16]:
eigvecs = np.array([ np.linalg.eig(sigma)[1] for sigma in paulis ])

In [17]:
proj_k = get_projectors(measures_test)

In [20]:
proj_k[0][0]

array([[0.5+0.j , 0. +0.5j, 0. +0.j , 0. +0.j ],
       [0. -0.5j, 0.5+0.j , 0. -0.j , 0. +0.j ],
       [0. +0.j , 0. +0.j , 0. +0.j , 0. +0.j ],
       [0. -0.j , 0. +0.j , 0. -0.j , 0. +0.j ]])

In [18]:
# Each measurement setting defines a new basis which is some multinomial distribution
probabilities = sampling_operator(rho, proj_k)
samples = np.array([ np.random.multinomial(N, basis_probs) for basis_probs in probabilities ])

# Normalize and flatten
Y = samples.flatten() / N

In [150]:

"""
# Parameters
d = 32  
r = 4
n = 1000
eta = 0.1 
epsilon = 1e-5 # convergence tolerance

# Generate data
X = np.random.randn(n, d)
S = np.random.randn(d, r) @ np.random.randn(r, d)
y = X @ S @ X.T  

# Initialize
U = np.random.randn(d,r)

prev_loss = float("inf")

while True:

    for i in range(n):
    
    # Get sample
        x = X[i,:]
    
    # Compute stochastic gradient    
        g = np.outer(x, x) 
        g /= np.sqrt(x.T @ U @ U.T @ x)

    # SGD update
        U = (np.eye(d) - eta*g) @ U

  # Compute loss
        loss = 0.5/n * np.linalg.norm(np.sqrt(y) - np.sqrt(X @ U @ U.T @ X.T))  

  # Check convergence
        if np.abs(prev_loss - loss) < epsilon:
            break
  
        prev_loss = loss"""

'\n# Parameters\nd = 32  \nr = 4\nn = 1000\neta = 0.1 \nepsilon = 1e-5 # convergence tolerance\n\n# Generate data\nX = np.random.randn(n, d)\nS = np.random.randn(d, r) @ np.random.randn(r, d)\ny = X @ S @ X.T  \n\n# Initialize\nU = np.random.randn(d,r)\n\nprev_loss = float("inf")\n\nwhile True:\n\n    for i in range(n):\n    \n    # Get sample\n        x = X[i,:]\n    \n    # Compute stochastic gradient    \n        g = np.outer(x, x) \n        g /= np.sqrt(x.T @ U @ U.T @ x)\n\n    # SGD update\n        U = (np.eye(d) - eta*g) @ U\n\n  # Compute loss\n        loss = 0.5/n * np.linalg.norm(np.sqrt(y) - np.sqrt(X @ U @ U.T @ X.T))  \n\n  # Check convergence\n        if np.abs(prev_loss - loss) < epsilon:\n            break\n  \n        prev_loss = loss'

In [151]:
"""def gradient_descent(proj_k):
    
    eta = 0.05
    s = np.zeros((4,4)) + np.zeros((4,4))*1j
    U = np.random.randn(4, 4)#init U
    for i in range(len(proj_k)):
        for j in range(len(proj_k[i])):
            num = ((proj_k[i][j] @ proj_k[i][j].conj().T) @ U)
            num = num / np.linalg.norm(U.T @ proj_k[i][j])
            s += num
            s = s * (eta/len(proj_k))
    U = (1-eta)*U - s"""

'def gradient_descent(proj_k):\n    \n    eta = 0.05\n    s = np.zeros((4,4)) + np.zeros((4,4))*1j\n    U = np.random.randn(4, 4)#init U\n    for i in range(len(proj_k)):\n        for j in range(len(proj_k[i])):\n            num = ((proj_k[i][j] @ proj_k[i][j].conj().T) @ U)\n            num = num / np.linalg.norm(U.T @ proj_k[i][j])\n            s += num\n            s = s * (eta/len(proj_k))\n    U = (1-eta)*U - s'

In [152]:
"U = np.random.randn(4, 4)"

'U = np.random.randn(4, 4)'

In [153]:
"U"

'U'

In [154]:
"""((proj_k[0][0] @ proj_k[0][0].conj().T) @ U).shape"""

'((proj_k[0][0] @ proj_k[0][0].conj().T) @ U).shape'

In [155]:
"""gradient_descent(proj_k)
eta = 0.05
    s = np.zeros((4,4)) + np.zeros((4,4))*1j
    U = np.random.randn(4, 4)#init U
    for i in range(len(proj_k)):
        for j in range(len(proj_k[i])):
            num = ((proj_k[i][j] @ proj_k[i][j].conj().T) @ U)
            num = num / np.linalg.norm(U.T @ proj_k[i][j])
            s += num
            s = s * (eta/len(proj_k))
    U = (1-eta)*U - s"""

'gradient_descent(proj_k)\neta = 0.05\n    s = np.zeros((4,4)) + np.zeros((4,4))*1j\n    U = np.random.randn(4, 4)#init U\n    for i in range(len(proj_k)):\n        for j in range(len(proj_k[i])):\n            num = ((proj_k[i][j] @ proj_k[i][j].conj().T) @ U)\n            num = num / np.linalg.norm(U.T @ proj_k[i][j])\n            s += num\n            s = s * (eta/len(proj_k))\n    U = (1-eta)*U - s'

In [156]:
eta = 0.005
s = np.zeros((4,4)) + np.zeros((4,4))*1j
U = np.random.randn(4, 4)#init U
U_ = np.random.randn(4, 4)
epsilon = 10e-10
steps = 0
while (np.linalg.norm(U_ - U) > epsilon and steps < 10000):
    prev_dist = np.linalg.norm(U_ - U)
    steps+=1
    U_ = U
    for i in range(len(proj_k)):
        for j in range(len(proj_k[i])):
            num = ((proj_k[i][j] @ proj_k[i][j].conj().T) @ U)
            num = num / np.linalg.norm(U.T @ proj_k[i][j])
            s += num
            s = s * (eta/len(proj_k))
    U = (1-eta)*U - s
    if prev_dist < np.linalg.norm(U - U_):
        eta /= 2

In [157]:
U

array([[ 5.16780484e-03+5.04365303e-09j,  1.23500456e-03-2.76684331e-09j,
         3.37062427e-03-1.20713125e-09j,  1.39469880e-03+2.88588157e-09j],
       [ 7.82856958e-03-3.32942239e-09j, -4.29459091e-03-7.95667052e-10j,
        -1.87366441e-03-2.17156669e-09j,  4.47935704e-03-8.98552294e-10j],
       [ 9.03981949e-04-1.11396072e-04j,  9.38540385e-05-5.05772559e-04j,
        -2.08657279e-03-4.34133478e-03j,  6.97956415e-04-2.14116599e-03j],
       [-1.11406927e-04-9.03893874e-04j, -5.05821841e-04-9.38448944e-05j,
        -4.34175779e-03+2.08636949e-03j, -2.14137462e-03-6.97888414e-04j]])

In [158]:
U_

array([[ 5.16785531e-03+5.04370228e-09j,  1.23501662e-03-2.76687033e-09j,
         3.37065719e-03-1.20714303e-09j,  1.39471242e-03+2.88590975e-09j],
       [ 7.82864604e-03-3.32945490e-09j, -4.29463285e-03-7.95674822e-10j,
        -1.87368270e-03-2.17158789e-09j,  4.47940078e-03-8.98561069e-10j],
       [ 9.03992690e-04-1.11397395e-04j,  9.38551537e-05-5.05778563e-04j,
        -2.08659758e-03-4.34138631e-03j,  6.97964709e-04-2.14119141e-03j],
       [-1.11408251e-04-9.03904605e-04j, -5.05827851e-04-9.38460084e-05j,
        -4.34180938e-03+2.08639426e-03j, -2.14140007e-03-6.97896699e-04j]])

In [159]:
np.linalg.norm(U-U_)

1.4865556910103759e-07

In [160]:
steps

10000